In [1]:
'''
changed (comment out) http://localhost:8888/edit/workplace/mxnet/python/mxnet/module/base_module.py
line 144 411 412
'''


import mxnet as mx
import numpy as np
out_dim = 6
compute_size = 128
D1 = mx.sym.Variable('data1')
S1 = mx.sym.Variable('s1',init = mx.init.Plusminusone(),shape = (1,20))
H1 = mx.sym.Variable('h1',init = mx.init.Index(out_dim),shape = (1,20))
D2 = mx.sym.Variable('data2')
S2 = mx.sym.Variable('s2',init = mx.init.Plusminusone(),shape = (1,20))
H2 = mx.sym.Variable('h2',init = mx.init.Index(out_dim),shape = (1,20))

cs1 = mx.sym.CountSketch( data = D1,s=S1, h = H1 ,name='cs1',out_dim = out_dim) 
cs2 = mx.sym.CountSketch( data = D2,s=S2, h = H2 ,name='cs2',out_dim = out_dim) 
fft1 = mx.sym.FFT(data = cs1, name='fft1', compute_size = compute_size) 
fft2 = mx.sym.FFT(data = cs2, name='fft2', compute_size = compute_size) 
c = fft1 * fft2
out = mx.sym.IFFT(data = c, name='ifft', compute_size = compute_size) 

mod = mx.mod.Module(symbol=out, 
                    context=mx.gpu(0),
                    data_names=['data1','data2'],
                    label_names = None
                    )

data = mx.io.NDArrayIter({'data1':np.random.uniform(-1, 1, (10,20)),'data2':np.random.uniform(-1, 1, (10,20))},batch_size=10,label_name=None)

from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

mod.bind(data_shapes=[('data1', (10,20)),('data2', (10,20))])
mod.init_params()
print mod.label_shapes
print mod.get_params()[0]['s1'].asnumpy()
print mod.get_params()[0]['h1'].asnumpy()
#mod.forward(Batch(data))
#mod.forward(Batch(data))

mod.fit(data, num_epoch=1)
print mod.get_outputs()[0].asnumpy()


None
[[-1.  1. -1.  1.  1. -1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1. -1. -1.
   1.  1.]]
[[ 1.  2.  1.  5.  3.  4.  5.  1.  0.  3.  1.  3.  5.  4.  0.  3.  3.  1.
   2.  2.]]
[[ -2.85288382   0.69883657   2.40398669  -0.91645956   2.40398669
    0.69883657]
 [  6.91815567  25.70142365   7.28585529  26.5063591    7.28585529
   25.70142365]
 [ -5.27399778   0.32251906  -5.11365604   0.36735582  -5.11365604
    0.32251906]
 [ 10.17026329   3.45135927  -5.43881607  -5.37584686  -5.43881655
    3.4513588 ]
 [-12.35376644   1.60196257 -10.04593849   8.21428394 -10.04593849
    1.60196257]
 [-16.04083443  -8.72570801  -5.62294388  -7.43520975  -5.62294388
   -8.72570801]
 [-16.39488983  16.33043289 -10.93972778  20.78601456 -10.93972778
   16.33043289]
 [ -6.18078518 -12.11275196  18.48058319  -9.09741879  18.48058319
  -12.11275196]
 [  6.85398769   4.71826506   8.28018951   8.81679916   8.28018951
    4.71826506]
 [-12.41288567   8.65018654   8.65702057 -23.72784996   8.65702057
    8.650